In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import math

In [ ]:
def visualize(scale, *images):
  figsize = ( images[0].shape[0] / max(images[0].shape[:2]) * scale, images[0].shape[1] / max(images[0].shape[:2]) * scale * len(images) )
  plt.figure(figsize=figsize)
  i = 1
  for id in range(len(images)):
    plt.subplot(1,len(images), i)
    plt.imshow(images[id])
    i += 1


img1 = cv.cvtColor(cv.imread('left_tr.jpg'), cv.COLOR_BGR2RGB)
img2 = cv.cvtColor(cv.imread('right.jpg'), cv.COLOR_BGR2RGB)
visualize(20, img1, img2)

In [ ]:
akaze = cv.AKAZE_create()
keypoints1, descriptors1 = akaze.detectAndCompute(img1, None)
keypoints2, descriptors2 = akaze.detectAndCompute(img2, None)

out1 = cv.drawKeypoints(img1, keypoints1, None, (255, 0, 255))
out2 = cv.drawKeypoints(img2, keypoints2, None, (255, 0, 255))
visualize(20, out1, out2)

In [ ]:
matcher = cv.BFMatcher_create(cv.NORM_L2)
matches = matcher.knnMatch(descriptors1, descriptors2,k=2)
all_matches = [m for m,n in matches]
out = cv.drawMatches(img1, keypoints1, img2, keypoints2, all_matches, None)
visualize(40, out)

In [ ]:
good = []
for m, n in matches:
    if m.distance < 0.6 * n.distance:
        good.append(m)

viz = cv.drawMatches(img1,keypoints1,img2,keypoints2,good,None)
visualize(60, viz)

In [ ]:
def warpImages(img1, img2, H):

  rows1, cols1 = img1.shape[:2]
  rows2, cols2 = img2.shape[:2]

  list_of_points_1 = np.float32([[0,0], [0, rows1],[cols1, rows1], [cols1, 0]]).reshape(-1, 1, 2)
  temp_points = np.float32([[0,0], [0,rows2], [cols2,rows2], [cols2,0]]).reshape(-1,1,2)

  list_of_points_2 = cv.perspectiveTransform(temp_points, H)

  list_of_points = np.concatenate((list_of_points_1,list_of_points_2), axis=0)
  
  [x_min, y_min] = np.int32(list_of_points.min(axis=0).ravel() - 0.5)
  [x_max, y_max] = np.int32(list_of_points.max(axis=0).ravel() + 0.5)
  
  translation_dist = [-x_min,-y_min]
  
  H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])

  output_img = cv.warpPerspective(img2, H_translation.dot(H), (x_max-x_min, y_max-y_min))
  output_img[translation_dist[1]:rows1+translation_dist[1], translation_dist[0]:cols1+translation_dist[0]] = img1

  return output_img

In [ ]:
MIN_MATCH_COUNT = 10

if len(good) > MIN_MATCH_COUNT:
    src_pts = np.float32([ keypoints1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    dst_pts = np.float32([ keypoints2[m.trainIdx].pt for m in good]).reshape(-1,1,2)

    M, _ = cv.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    
    result = warpImages(img2, img1, M)

visualize(30, result)